In [1]:
from numba import cuda, jit, float32
import numpy as np
import matplotlib.pyplot as plt

# Short test about indexing...

In [19]:
TPB = 10

@cuda.jit
def indexing(A, x, b, out):
    sA = cuda.shared.array(shape=(10), dtype=float32)

    tx = cuda.threadIdx.x
    bx = cuda.blockIdx.x

    if tx < 10:
        tmp = 0.
        
        sA[tx] = inner_product_for_grad(A[tx + bx * TPB,:], x, b[tx + bx * TPB])
        
        tmp = sA[tx]

        out[tx+bx*TPB] = tmp

In [20]:
A = np.random.rand(20,20)
x = np.random.rand(20)
b = np.random.rand(20)
out = np.zeros((20))
indexing[(2,2),(TPB,TPB)](A, x, b, out)
print(out)

[4.15543365 4.14736128 3.8928833  4.27884531 3.36535811 3.54664683
 4.19189072 3.31274247 3.9562645  4.73452187 4.5501895  4.39229393
 3.78884864 2.59125161 4.16585445 2.2434299  4.46062183 4.18280029
 2.0414443  3.04799104]


In [21]:
(A @ x - b)

array([4.15543382, 4.14736121, 3.8928832 , 4.27884542, 3.36535819,
       3.54664694, 4.19189072, 3.31274248, 3.95626442, 4.73452186,
       4.55018964, 4.39229415, 3.78884876, 2.59125173, 4.16585438,
       2.24342982, 4.46062193, 4.18280051, 2.04144427, 3.04799097])

# Test

In [4]:
@jit(nopython=True)
def inner_product_for_grad(x, y, b):
    out = 0.
    
    for i in range(x.size):
        out += x[i] * y[i]
    
    out -= b

    return out
    
TPB = 16

@cuda.jit
def optimizer(A, x, b, lr, out):
    sA = cuda.shared.array(shape=(TPB,TPB), dtype=float32)
    sB = cuda.shared.array(shape=(TPB), dtype=float32)

    i = cuda.grid(1)

    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y

    if i < x.shape[0]:
        ## Quit if (x) is outside of valid out boundary
        
        for j in range(int(A.shape[0] / TPB)):
            tmp = 0.    

            ## Preload data into shared memory
            sA[tx, ty] = A.T[i, ty + j * TPB]
            sB[tx+j] = inner_product_for_grad(A[tx + j * TPB,:], x, b[tx + j * TPB])

            ## Wait until all threads finish proloading
            cuda.syncthreads()

            ## Computes partial product on the shared memory
            for k in range(TPB):
                tmp += sA[tx, k] * sB[k]

            ## Wait until all threads finish computing
            cuda.syncthreads()

        out[i] = tmp * 2

In [ ]:
A = np.random.rand(1000,1000)
b = np.random.rand(1000)
x = np.random.rand(1000)
out = np.zeros((1000))

A_ = cuda.to_device(A)
b_ = cuda.to_device(b)
x_ = cuda.to_device(x)
out_ = cuda.to_device(out)

lr = 1e-6

In [ ]:
grad = 2 * A.T @ (A @ x - b)
x -= grad * lr

## Configure the blocks
threadsperblock = (TPB,TPB)
blockspergrid_x = int(np.ceil(A.shape[0] / threadsperblock[1]))
blockspergrid_y = int(np.ceil(A.shape[1] / threadsperblock[0]))
blockspergrid = (blockspergrid_x, blockspergrid_y)

optimizer[blockspergrid, threadsperblock](A_, x_, b_, lr, out_)
## cuda.synchronize()

In [ ]:
np.linalg.norm(out_ - grad)

12559831.725702958

In [ ]:
out_[:10].copy_to_host() - grad[:10]

array([-242814.84076504, -240358.80777922,  120106.65612123,
       1314363.88965597, -246367.13676416, -242396.51960289,
       -240911.87850293, -234407.09545978, -239276.99704383,
       -247892.68513988])

In [ ]:
## Using one GPU 
class LeastSquare():
    def __init__(self, A, b, epoches=10, TPB=16):
        self.A = A
        self.b = b
        self.lr = 1e-3/A.shape[1]
        self.epoches = epoches
        self.x = cuda.to_device(np.random.rand(A.shape[1]))
        self.x_hat = cuda.device_array((A.shape[1]))
        self.error_list = []
        self.grad = cuda.device_array((A.shape[1]))

        ## About kernel, Configure the blocks
        self.threadsperblock = (TPB,TPB) 
        blockspergrid_x = int(np.ceil(A.shape[0] / self.threadsperblock[1]))
        blockspergrid_y = int(np.ceil(A.shape[1] / self.threadsperblock[0]))
        self.blockspergrid = (blockspergrid_x, blockspergrid_y)
        
    def run(self):
        for i in range(self.epoches):
            A, b = self.initialize()
            self.optimize(A, b, self.x)

        return self.x_hat

    def initialize(self):
        index = np.random.choice(self.A.shape[0], 1000)
        A = cuda.to_device(self.A[index,:])
        b = cuda.to_device(self.b[index])

        return A, b

    def optimize(self, A, b, x, iters_per_epoch=500):
        
        for i in range(iters_per_epoch):
            optimizer[self.blockspergrid, self.threadsperblock](A, x, b, lr)

    def check(self, x):
        b_hat = self.A @ x
        error = np.linalg.norm(self.b - b_hat)
        self.error_list.append(error)

        return error

In [ ]:
lstsq = LeastSquare(A, b)
x_hat = lstsq.run()
x_hat_cpu = x_hat.copy_to_host()
lstsq.check(x_hat_cpu)